In [190]:
import pandas as pd
import time
import numpy as np
import re
import json
import csv

# 0. Understanding the dataset

Before starting any kind of data analysis, we first have to review the content of our dataset and understand its meaning better. To do this, we use the data sample and the schema provided with the dataset.

## 0.1. Dataset structure
Let us first read the `schema.txt` file to understand what fields our dataset contains.

In [186]:
schema = pd.read_table('twitter-swisscom/schema.txt', delimiter='    ', engine='python',
                       names=['Field name', 'Type', 'Specification', 'Unsigned', 'Optional field', 'Format'] )
schema

,Field name,Type,Specification,Unsigned,Optional field,Format
1,id,bigint(20),NaN,UNSIGNED,No,None
2,userId,bigint(20),NaN,UNSIGNED,No,None
3,createdAt,timestamp,NaN,NaN,No,0000-00-00 00:00:00
4,text,text,utf8_unicode_ci,NaN,No,None
5,longitude,float,NaN,NaN,Yes,NaN
6,latitude,float,NaN,NaN,Yes,NaN
7,placeId,varchar(25),utf8_general_ci,NaN,Yes,NaN
8,inReplyTo,bigint(20),NaN,UNSIGNED,Yes,NaN
9,source,int(10),NaN,UNSIGNED,No,None
10,truncated,bit(1),NaN,NaN,No,None


The `schema.txt` contains information about each field in our dataset. We see that there are 20 possible fields for a tweet, but many of them are optional, so most tweets probably haver fewer specified field than that.

Here are the fields that will be the most useful:
- userId: to know who posted the tweet
- createdAt: to know when the tweet was posted
- text: content of the tweet
- longitude and latitude: to know from where the tweet was posted


## 0.2. Dataset contents
Now, let us have a look at the actual content of our dataset using the provided sample.

In [206]:
sample_df = pd.read_csv('twitter-swisscom/sample.tsv', encoding='utf-8', sep='\t', escapechar='\\', 
                        index_col='id', names=schema['Field name'], quoting=csv.QUOTE_NONE, na_values='N')
sample_df.head()

,userId,createdAt,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation
id,,,,,,,,,,,,,,,,,,,
776522983837954049,735449229028675584,2016-09-15 20:48:01,se lo dici tu... https://t.co/x7Qm1VHBKL,NaN,NaN,51c0e6b24c64e54e,NaN,1,NaN,46.0027,8.96044,Twitter for iPhone,http://twitter.com/#!/download/iphone,plvtone filiae.,hazel_chb,146,110,28621,Earleen.
776523000636203010,2741685639,2016-09-15 20:48:05,https://t.co/noYrTnqmg9,NaN,NaN,4e7c21fd2af027c6,NaN,1,NaN,46.8131,8.22414,Twitter for iPhone,http://twitter.com/#!/download/iphone,samara,letisieg,755,2037,3771,Suisse
776523045200691200,435239151,2016-09-15 20:48:15,@BesacTof @Leonid_CCCP Tu dois t'engager en si...,NaN,NaN,12eb9b254faf37a3,7.765221e+17,5,NaN,47.2010,5.94082,Twitter for Android,http://twitter.com/download/android,lebrübrü❤,lebrubru,811,595,30191,Fontain
776523058404290560,503244217,2016-09-15 20:48:18,@Mno0or_Abyat اشوف مظاهرات على قانون العمل الج...,NaN,NaN,30bcd7f767b4041e,7.765216e+17,1,NaN,45.8011,6.16552,Twitter for iPhone,http://twitter.com/#!/download/iphone,عبدالله القنيص,bingnais,28433,417,12262,Shargeyah
776523058504925185,452805259,2016-09-15 20:48:18,Greek night #geneve (@ Emilios in Genève) http...,6.14414,46.1966,c3a6437e1b1a726d,NaN,3,NaN,46.2048,6.14319,foursquare,http://foursquare.com,Alkan Şenli,Alkanoli,204,172,3390,İstanbul/Burgazada


Let us check the percentage of `NaN`values for each field.

In [212]:
sample_df.isnull().count().apply()

userId            8790
createdAt         8790
text              8790
longitude         8790
latitude          8790
placeId           8790
inReplyTo         8790
source            8790
truncated         8790
placeLatitude     8790
placeLongitude    8790
sourceName        8790
sourceUrl         8790
userName          8790
screenName        8790
followersCount    8790
friendsCount      8790
statusesCount     8790
userLocation      8790
dtype: int64

We can observe that the `latitude`and `longitude` fields seem to `NaN` quite often, whereas the `placeLatitude`and `placeLongitude` fields are almost always specified.

# 1. Data cleaning and pre-processing

In [180]:
col = ['id', 'userId', 'createdAt', 'text', 'long', 'lat', 'placeId', 'inReplyTo', 'source', 'truncated', 'Longitude',\
       'Latitude', 'sourceName', 'sourceUrl', 'username', 'screenName', 'followerscount', 'friendscount', 'statusescount',\
       'userLocation']
keep_col = ['id', 'userId', 'createdAt', 'text', 'Longitude', 'Latitude', 'username']
keep_final = ['id', 'userId', 'createdAt', 'Longitude', 'Latitude', 'username', 'tag', 'at']

In [79]:
def get_hashtags(text):
    """Returns the list of all hashtags (e.g. '#hashtag') present in the given text"""
    try:
        res = re.findall(r"#\w+", text)
        return [s.lower() for s in res]
    except:
        print(text)
        return list()
    
def get_mentions(t):
    """Returns the list of all mentions (e.g. '@mention') present in the given text"""
    return re.findall(r"@\w+", t)

def add_lines_in_df(lines, dataframe):
    df2 =  pd.DataFrame(lines)
    df2.columns = col
    df2 = df2[keep_col]
    df2['tag'] = df2.text.apply(lambda t: get_hashtags(t))
    df2['at'] = df2.text.apply(lambda t: get_mentions(t))
    df2 = df2[keep_final]
    return pd.concat([dataframe, df2], ignore_index=True)

In [68]:
file = open("../../twitter_dataset/twitter_full.tsv", encoding="utf8")
data = file.readline()
j = 0
t = time.time()
data_list = list()
df = None
while not data == "":
    j+=1
    l = len(data.split('\t'))
    while l < 20:
        data += file.readline()
        l = len(data.split('\t'))
    if l == 20:
        data_list.append(data.split('\t'))
    if j%100000 == 0:
        print(j)
        df2 =  pd.DataFrame(data_list)
        df2.columns = col
        df2 = df2[keep_col]
        df2['tag'] = df2.text.apply(lambda x: keep_tags(x))
        df2['at'] = df2.text.apply(lambda x: keep_at(x))
        df2 = df2[keep_final]
        df = pd.concat([df, df2], ignore_index=True)
        data_list = list()
    
    data = file.readline()
    if data == "":
        print(j)
        df2 =  pd.DataFrame(d)
        df2.columns = col
        df2 = df2[keep_col]
        df2['tag'] = df2.text.apply(lambda x: keep_tags(x))
        df2['at'] = df2.text.apply(lambda x: keep_at(x))
        df2 = df2[keep_final]
        df = pd.concat([df, df2], ignore_index=True)
        d = list()
    

print(time.time()-t)
file.close()


100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000
1

In [147]:
df.count()

id           20212854
userId       20212854
createdAt    20212854
Longitude    20212854
Latitude     20212854
username     20157151
tag          20212854
dtype: int64

In [157]:
df[df['Longitude'] != '\\N'].count()

id           20191872
userId       20191872
createdAt    20191872
Longitude    20191872
Latitude     20191872
username     20136169
tag          20191872
dtype: int64

In [71]:
df.to_csv('../../twitter_dataset/cleaned.csv', sep=';')

In [67]:
df_tag = df[(df["tag"].astype(str) != '[]')]
df_tag.to_csv('../../twitter_dataset/cleaned_hashtag.csv', sep=';')

In [67]:
df_at = df[(df["at"].astype(str) != '[]')]
df_at.to_csv('../../twitter_dataset/cleaned_identification.csv', sep=';')

# 2. Grouping by hashtag

The first step of our descriptive data analysis is to group all tweets by hashtags. This will allow us to count find the most popular hashtags, and later agreagate them by time.

In [66]:
df = pd.read_csv("../../twitter_dataset/cleaned.csv", sep=';', encoding="cp437", usecols=range(1,8))

//anaconda/envs/ada/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [69]:
df_tag = pd.read_csv("../../twitter_dataset/cleaned_hashtag.csv", sep=';', encoding="cp437", usecols=range(1,8))

//anaconda/envs/ada/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [140]:
df_at = pd.read_csv("../../twitter_dataset/cleaned_identification.csv", sep=';', encoding="cp437", usecols=range(1,8))

//anaconda/envs/ada/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [158]:
def compute_unique_tag_list_and_store():
    """Compute the list of different hashtags"""
    
    concat_list = np.concatenate(df_tag.tag.apply(lambda x : np.array(keep_tags(x))))
    unique_tags = np.unique(concat_list)
    with open('../../twitter_dataset/unique_hashtags.json', 'w') as outfile:
        json.dump(unique_tags.tolist(), outfile)
    
def load_unique_tag_list():
    with open('../../twitter_dataset/unique_hashtags.json', 'r') as infile:
        unique_tags = json.load(infile)
    return unique_tags


In [ ]:
compute_unique_tag_list_and_store()

In [161]:
def get_index_with_hashtag(df, hashtag):
    return np.where(df.tag.apply(lambda x : ("'"+hashtag+"'") in x))

def group_by_hashtag(df):
    unique_tags = load_unique_tag_list()    
    output = {}
    for hashtag in unique_tags:
        output[hashtag] = get_index_with_hashtag(df, hashtag)
    output_df = pd.DataFrame.from_dict(output, orient='index')
    output_df.reset_index(inplace=True)
    output_df.columns = ['hashtag', 'tweets_idx']
    return output_df

group_hashtags = group_by_hashtag(df_tag.head(100))
group_hashtags

,hashtag,tweets_idx
0,#oneyearinaustralia,[]
1,#BayramHediyemizQUARESMA,[]
2,#ray,[]
3,#tibetaansegroet,[]
4,#YouR,[]
5,#Meynet,[]
6,#panefarcito,[]
7,#IFeelDown,[]
8,#Qrcode,[]
9,#lamiatesta,[]


In [139]:
group_hashtags_nb = group_hashtags.copy()
group_hashtags_nb['tweets_nb'] = group_hashtags_nb['tweets_idx'].apply(lambda ls: len(ls))
group_hashtags_nb.sort_values('tweets_nb', ascending=False).drop(['tweets_idx'], axis=1).head(15)

,hashtag,tweets_nb
467,#fb,962
5412,#fail,216
1390,#lift11,91
3990,#Endomondo,80
973,#bosw,66
6297,#android,65
4510,#sbb,64
633,#iphoneography,60
2335,#ff,58
2692,#esc,57
